## Resume Optimizer using Open AI

This Jupyter Notebook is designed to optimize resumes by leveraging AI models from OpenAI. The notebook guides you through the process of importing your resume and a job description, and then uses AI to tailor your resume to better match the job requirements. The key functionalities include:

1. **Library Installation and Imports**: Installs and imports necessary libraries such as `google-generativeai`, `markdown`, and `weasyprint`.
2. **Resume and Job Description Input**: Prompts the user to input their resume and job description.
3. **Prompt Template**: Builds a prompt template to instruct the AI on how to optimize the resume.
4. **Model Interaction**: Lists available AI models and uses a selected model to generate optimized resume content.
5. **Output Generation**: Displays the optimized resume in Markdown format and provides suggestions for improvement.
6. **File Saving**: Saves the optimized resume as Markdown, HTML, and PDF files.

This notebook aims to help users create a more targeted and effective resume by aligning it with specific job descriptions, enhancing their chances of success in job applications.


**Shout out to [Shaw Tabeli](https://www.shawhintalebi.com/) for the solid foundation and inspiration**

In [ ]:
# Install Libs
!pip install openai markdown weasyprint

**Import the Libraries**

In [5]:
from IPython.display import display, Markdown
import os
import openai
from markdown import markdown
from weasyprint import HTML

# Connect to OpenAI
openai.api_key = os.getenv("OPENAI_KEY")

### Import Resume and Job Description

In [ ]:
# Open and Read the resume.md 
with open("resume/morrison_resume.md", "r", encoding="utf-8") as file:
    resume_string = file.read()

display(Markdown(resume_string))

In [ ]:
# Prompt user for job data
jd_string = "" 
while not jd_string:  # Loop until we get some input
    print("To help me analyze this job description for you, please paste the full text below:")
    jd_string = input()
    print(jd_string)
    if not jd_string:
        print("Oops! Looks like you didn't paste anything. Please try again.")
        
print(jd_string)

Build the Prompt Template

In [25]:
prompt_template = lambda resume_string, jd_string : f"""
You are a professional resume optimization expert specializing in tailoring resumes to specific job descriptions. Your goal is to optimize my resume and provide actionable suggestions for improvement to align with the target role.

### Guidelines:
1. **Relevance**:  
   - Prioritize experiences, skills, and achievements **most relevant to the job description**.  
   - Remove or de-emphasize irrelevant details to ensure a **concise** and **targeted** resume.
   - Limit work experience section to 4-5 most relevant roles
   - Limit bullet points under each role to 4-5 most relevant impacts

2. **Action-Driven Results**:  
   - Use **strong action verbs** and **quantifiable results** (e.g., percentages, revenue, efficiency improvements) to highlight impact.  

3. **Keyword Optimization**:  
   - Integrate **keywords** and phrases from the job description naturally to optimize for ATS (Applicant Tracking Systems).  

4. **Additional Suggestions** *(If Gaps Exist)*:  
   - If the resume does not fully align with the job description, suggest:  
     1. **Additional technical or soft skills** that I could add to make my profile stronger.  
     2. **Certifications or courses** I could pursue to bridge the gap.  
     3. **Project ideas or experiences** that would better align with the role.  

5. **Formatting**:  
   - Output the tailored resume in **clean Markdown format**.  
   - Include an **"Additional Suggestions"** section at the end with actionable improvement recommendations.  
   - Aim for a resume length of around 1000-1500 words.
   - Tailor the tone and style to match the target company and industry.

---

### Input:
- **My resume**:  
{resume_string}

- **The job description**:  
{jd_string}

---

### Output:  
 **Tailored Resume**:  
   - A resume in **Markdown format** that emphasizes relevant experience, skills, and achievements.  
   - Incorporates job description **keywords** to optimize for ATS.  
   - Uses strong language and is no longer than **two pages**.

 **Contact Information:**  <-- Separate section for contact info
    - Include all provided contact details (email, phone, LinkedIn, GitHub).

 **Certifications:**  <-- Separate section for certifications at the bottom
    - Include all provided certifications with links.

## Addtional Suggestions:  <-- Separate section for suggestions
    - Include any additional suggestions for improvement based on the job description.
 **Additional Suggestions** *(if applicable)*:
   - Determine if the resume and experience is a **FIT** and a good candidate for this specific role.
   - List **skills** that could strengthen alignment with the role.  
   - Recommend **certifications or courses** to pursue.  
   - Suggest **specific projects or experiences** to develop.
   - Do not include "```markdown\n" or "```" in the response.

 **Evaluate as if you are the hiring manager the resume and experience against the job description to determine if the candidate is:
   - **FIT**: Fully meets or exceeds requirements.
   - **CLOSE**: Meets most requirements with minor gaps.
   - **STRETCH**: Has significant gaps but potential to grow into the role.

"""

In [ ]:
# List all available models
models = openai.Model.list()

# Print the names of the models
for model in models['data']:
    print(model['id'])

In [26]:
prompt = prompt_template(resume_string, jd_string)

In [27]:
# Make API call
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Expert resume writer"},
        {"role": "user", "content": prompt}
    ],
    temperature=0.5
)

# Extract the response content
generated_text = response.choices[0].message.content

### Display the results
Split the resume from the Additional suggestions

In [28]:
resume_list = generated_text.split("## Additional Suggestions")

In [ ]:
print(resume_list[0])

Display the resume

In [ ]:
display(Markdown(resume_list[0]))

### Save File to PDF & HTML

In [ ]:
# Save to PDF
output_resume_pdf = "resume/openai_resume_new.pdf"

# Convert Markdown to HTML
html_output = markdown(resume_list[0])
print(html_output)

In [ ]:
from markdown import markdown

def markdown_to_html_with_css(markdown_text, css_file="styles.css", google_font="Roboto"):
  """
  Converts markdown text to HTML, applies an external CSS file, and adds a Google Font.

  Args:
    markdown_text: The markdown text to convert.
    css_file: The path to the CSS file.
    google_font: The name of the Google Font to use.

  Returns:
    The HTML output as a string.
  """

  html_output = markdown(markdown_text)

  # Construct the HTML with the linked CSS file and Google Font
  html_with_css = f"""
  <!DOCTYPE html>
  <html>
  <head>
    <meta charset="UTF-8">
    <title>Resume</title>
    <link rel="stylesheet" href="{css_file}">
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family={google_font}&display=swap" rel="stylesheet">
  </head>
  <body>
    {html_output}
  </body>
  </html>
  """

  return html_with_css

# Example usage with resume_list and a Google Font:
html_output = markdown_to_html_with_css(resume_list[0], google_font="Open+Sans")

# Now you can save html_output to a file or use it further
print(html_output)

# Specify the filename
filename = "resume/openai_resume.html"

# Write the HTML to the file
with open(filename, "w") as f:
  f.write(html_output)

print(f"HTML file saved as {filename}")

In [33]:
# Save to PDF
output_resume_pdf = "resume/openai_resume_new.pdf"

# Convert HTML to PDF and save
HTML(string=html_output).write_pdf(output_resume_pdf, stylesheets=['resume/styles.css'])

In [34]:
# save as markdown
output_file = "resume/openai_resume_new.md"

with open(output_file, "w", encoding="utf-8") as file:
    file.write(resume_list[0])

In [ ]:
display(Markdown(resume_list[1]))